# Advantage Actor Critic

In [ ]:
import gym
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

# **Auswahl des Spiels**

[Hier](https://gym.openai.com/envs/#atari) ist eine vollständige Liste der verfügbaren Spiele zu finden. Um ein Environment zu erstellen muss der vollständige Name des Spiels als String übergeben werden.

Beispiel:

```python
game = "MsPacman-v0"
```

In [ ]:
# Hier kann das Spiel übergeben werden
game = "Breakout-v0"

Der folgende Schritt ist notwendig, wenn das Notebook in Google Colab ausgeführt wird. Je nach installierter Version von OpenAIs Gym Bibliothek müssen die Atari-Spiele importiert werden, damit sie in Gym verfürbar sind. Die Spiele können [Hier](http://www.atarimania.com/rom_collection_archive_atari_2600_roms.html) heruntergeladen werden und müssen anschließend in Google Drive hinterlegt werden.

In [ ]:
try: 
    env = gym.make(game)
except:
    from google.colab import drive
    drive.mount('/content/drive')
    !python -m atari_py.import_roms "drive/MyDrive/Atari_Roms"
    env = gym.make(game)

# **Preprocessing**
Das Preprocessing der Daten ist enorm wichtig, da es einen großen Einfluss auf den Lernprozess des Agenten am Ende des Tages hat.
Die Bibliothek Gym stellt hierfür Wrapper bereit, die die verschiedenen Spiele um Funktionalitäten erweitern können und das Lernen optimieren.

Unter [Stable Baselines](https://github.com/openai/baselines/blob/master/baselines/common/atari_wrappers.py) sind schon vordefinierte Wrapper implementiert, die zum Training des Agenten nützlich sind. Die von "Stable Baselines" übernommenen Wrapper werden im folgenden kurz erläutert.

### Fire Wrapper
Der FireResetEnv Wrapper ist dafür zuständig, dass die "FIRE"-Aktion bei Spielreset ausgeführt wird. In manchen Spielen ist dies notwendig, damit das Spiel startet. Der Wrapper fragt zuvor die gültigen Aktionen ab und führt "FIRE" nur aus, wenn diese sich im Aktionsraum befindet.

In [ ]:
class FireResetEnv(gym.Wrapper):
    def __init__(self, env):
        gym.Wrapper.__init__(self, env) 
        self.env.reset()

    def reset(self, **kwargs):
        self.env.reset(**kwargs)
        observation, _, _, _ = self.env.step(env.unwrapped.get_action_meanings().index('FIRE'))

        return observation

### MaxAndSkip Wrapper
Aufeinanderfolgende Zustände in Atari-Spielen können schwer voneinander zu unterscheiden sein, da diese in kurzen Zeitabständen aufeinander folgen. Die Änderungen können so klein sein, dass diese für das Verhalten des Agenten nicht relevant erscheinen und deswegen übersprungen werden können. Der MaxAndSkipEnv Wrapper führt für N aufeinanderfolgende Aktionen die gleiche Aktion aus, da die Änderungen in diesem Zeitraum zu klein erscheinen.

In [ ]:
class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env, skip=4):
        """Return only every `skip`-th frame"""
        gym.Wrapper.__init__(self, env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = np.zeros((2,)+env.observation_space.shape, dtype=np.uint8)
        self._skip       = skip

    def step(self, action):
        """Repeat action, sum reward, and max over last observations."""
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            if i == self._skip - 2: self._obs_buffer[0] = obs
            if i == self._skip - 1: self._obs_buffer[1] = obs
            total_reward += reward
            if done:
                break
        # Note that the observation on the done=True frame
        # doesn't matter
        max_frame = self._obs_buffer.max(axis=0)

        return max_frame, total_reward, done, info
    
    def reset(self, **kwargs):
        return self.env.reset(**kwargs)

### ScaledFloatFrame Wrapper
Künstliche neuronale Netze lernen effektiver, wenn sich die eingegbenen Werte in einem Intervall von 0 bis 1 oder -1 bis 1 befinden. Der ScaledFloatFrame Wrapper skalliert die Bilder auf ein vom Benutzer festgelegtes Intervall herunter. Das Intervall kann mit den Parametern "low" und "high" festgelegt werden.

In [ ]:
class ScaledFloatFrame(gym.ObservationWrapper):
    def __init__(self, env):
        gym.ObservationWrapper.__init__(self, env)
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=env.observation_space.shape, dtype=np.float32)

    def observation(self, observation):
        # careful! This undoes the memory optimization, use
        # with smaller replay buffers only.
        return np.array(observation).astype(np.float32) / 255.0

### EpisodicLifeEnv Wrapper
Manche Atari-Spiele lassen es zu, dass der Agent nach einer gescheiterten Aktion von einem Checkpoint aus startet. Das ermöglicht es ihm auch fortgeschrittene Zustände zu betreten und zu sampeln. Es kann aber auch vorkommen, dass er fehlerhafte Aktionen mit in das Lernen aufnimmt, da er so in fortgeschrittene Zustände kommt. Bei dem Spiel Breakout-v0 kommt es häufig vor, dass der Agent die ersten Bälle durchlässt, bevor er agiert. Der Agent positioniert sich dabei an einen Spielfeldrand, da der Spielball ab einen bestimmten Zeipunkt automatisch dort abgeworfen wird. Erst dann beginnt der Agent richtig zu spielen.
Dieses Verhalten ist aber nicht immer erwünscht, deshalb kann der EpisodicLifeEnv Wrapper eingesetzt werden. Dieser Wrapper verhindert das Starten aus Checkpoints heraus und startet das Spiel neu sobald der Agent eine fehlerhafte Aktion begangen hat.
Dies kann aber auch zum Nachteil haben, dass der Agent erst spät fortgeschrittene Spielzustände betritt und das Training zum angestrebten Leistungsziel verlängert wird.

In [ ]:
class EpisodicLifeEnv(gym.Wrapper):
    def __init__(self, env):
        """Make end-of-life == end-of-episode, but only reset on true game over.
        Done by DeepMind for the DQN and co. since it helps value estimation.
        """
        gym.Wrapper.__init__(self, env)
        self.lives = 0
        self.was_real_done  = True

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        self.was_real_done = done
        # check current lives, make loss of life terminal,
        # then update lives to handle bonus lives
        lives = self.env.unwrapped.ale.lives()
        if lives < self.lives and lives > 0:
            # for Qbert sometimes we stay in lives == 0 condition for a few frames
            # so it's important to keep lives > 0, so that we only reset once
            # the environment advertises done.
            done = True
        self.lives = lives
        return obs, reward, done, info

    def reset(self, **kwargs):
        """Reset only when lives are exhausted.
        This way all states are still reachable even though lives are episodic,
        and the learner need not know about any of this behind-the-scenes.
        """
        if self.was_real_done:
            obs = self.env.reset(**kwargs)
        else:
            # no-op step to advance from terminal/lost life state
            obs, _, _, _ = self.env.step(0)
        self.lives = self.env.unwrapped.ale.lives()
        return obs

### Clip Reward Wrapper
Der ClipRewardEnv Wrapper beschneidet den Reward auf das Intervall von -1 bis 1. Dieses Verhalten kann erwünscht sein, wenn Rewards nicht zu stark von der Umgebung bewertet werden und der Agent vollständig aus eigenen Erfahrungen lernt die Zustände zu bewerten.

In [ ]:
class ClipRewardEnv(gym.RewardWrapper):
    def __init__(self, env):
        gym.RewardWrapper.__init__(self, env)

    def reward(self, reward):
        """Bin reward to {+1, 0, -1} by its sign."""
        return np.sign(reward)

### Resize & Grayscale Wrapper
Der WarpFrame Wrapper nimmt Einfluss auf die Beobachtung, die der Agent von der Umgebung erhält.
Die Ausgangsbilder werden auf Bilder der Größe 84x84 Pixel herunterskaliert und anschleßend auf einen Farbkanal reduziert. Die Farben sind für das Lernen des Spiels nicht wichtig, sodass diese im Sinne einer besseren Rechenkapazität ausgelassen werden können.

In [ ]:
from gym import spaces
import cv2

class WarpFrame(gym.ObservationWrapper):
    def __init__(self, env):
        """
        Warp frames to 84x84 as done in the Nature paper and later work.
        :param env: (Gym Environment) the environment
        """
        gym.ObservationWrapper.__init__(self, env)
        self.width = 84
        self.height = 84
        self.observation_space = spaces.Box(low=0, high=255, shape=(self.height, self.width, 1),
                                            dtype=env.observation_space.dtype)
        
    def observation(self, frame):
        """
        returns the current observation from a frame
        :param frame: ([int] or [float]) environment frame
        :return: ([int] or [float]) the observation
        """
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(frame, (self.width, self.height), interpolation=cv2.INTER_AREA)
        return frame[:, :, None]

Die folgende Codezelle ist optional ausführbar und instanziirt eine Umgebung beispielhaft, um den Effekt des WarpFrame Wrapper darzustellen.

In [ ]:
""" 
[OPTIONAL]

Diese Zelle ist optional ausführbar und dient zur Visualisierung des Wrappers.
Die Zelle hat keinen Einfluss auf den Agenten
"""

def WarpFrameEnv(env_name):
    env = gym.make(env_name)
    env = WarpFrame(env)
    return env

normal_env = gym.make(game)
wrapped_env = WarpFrameEnv(game)

normal_env.reset()
wrapped_env.reset()
action = normal_env.action_space.sample()

normal_state, _, _, _ = normal_env.step(action)
wrapped_state, _, _, _ = wrapped_env.step(action)

wrapped_state = wrapped_state[: , :, 0]

fig, axs = plt.subplots(1, 2)
fig.suptitle('Warp Frame', fontsize=20)
axs[0].imshow(normal_state)
axs[0].set_title("Normal", fontsize=16)
axs[1].imshow(wrapped_state, cmap="gray")
axs[1].set_title("Warp Frame", fontsize=16)
plt.show()

### Frame Stack Wrapper
Der FrameStack Wrapper stapelt N aufeinanderfolgende Zustände aufeinander, sodass diese dem Agenten zusammen als ein Zustand übergeben werden können.
Das Stapeln der Zustände ist hilfreich, da der Agent so Bewegungen in der Umgebung wahrnehmen kann.

In [ ]:
from collections import deque

class FrameStack(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
        self.frames = deque(maxlen=4)
        low = np.repeat(self.observation_space.low[np.newaxis, ...], repeats=4, axis=0)
        high = np.repeat(self.observation_space.high[np.newaxis, ...], repeats=4, axis=0)
        self.observation_space = gym.spaces.Box(low=low, high=high, dtype=self.observation_space.dtype)

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        self.frames.append(obs)
        frame_stack = np.asarray(self.frames, dtype=np.float32)
        frame_stack = np.moveaxis(frame_stack, 0, -1).reshape(1, 84, 84, -1)
        return frame_stack, reward, done, info

    def reset(self, **kwargs):
        obs = self.env.reset(**kwargs)
        for _ in range(4):
            self.frames.append(obs)
        frame_stack = np.asarray(self.frames, dtype=np.float32)
        frame_stack = np.moveaxis(frame_stack, 0, -1).reshape(1, 84, 84, -1)
        return frame_stack

Die folgende Codezelle ist optional ausführbar und instanziirt eine Umgebung beispielhaft, um den Effekt des FrameStack Wrapper darzustellen.

In [ ]:
"""
[OPTIONAL]

Diese Zelle ist optional ausführbar und dient zur Visualisierung des Wrappers.
Die Zelle hat keinen Einfluss auf den Agenten
"""

def FrameStackEnv(env_name):
    env = gym.make(env_name)
    env = WarpFrame(env)
    if 'FIRE' in env.unwrapped.get_action_meanings():
        env = FireResetEnv(env)
    env = FrameStack(env)
    return env

env = FrameStackEnv(game)
env.reset()

for _ in range(1, 5):
  # Führe eine zufällige Aktion aus
  state, _, _, _ = env.step(env.action_space.sample())

# Stack umformen, damit das Plotten der vier Bilder gelingt
state = state.reshape(84, 84,4)

# Frame Stack plotten
fig, axs = plt.subplots(1,4, figsize=(15, 5))
fig.suptitle('Frame Stack', fontsize=20)
for i in range(state.shape[2]):
    axs[i].imshow(state[:, :, i], cmap="gray")
    axs[i].set_title("Frame "+str(i+1), fontsize=16)
plt.show()

### Erstellen des Environments
Hier werden die gewählten Wrapper an die Umgebung gebunden und diese anschließen instanziiert.

In [ ]:
def make_env(env_name):
    env = gym.make(env_name)
    #env = EpisodicLifeEnv(env)
    env = MaxAndSkipEnv(env)
    env = ScaledFloatFrame(env)
    env = NoopResetEnv(env)
    #env = ClipRewardEnv(env)
    env = WarpFrame(env)
    if 'FIRE' in env.unwrapped.get_action_meanings():
        env = FireResetEnv(env)
    env = FrameStack(env)
    return env

env = make_env(game)

""" saving the properties for csv """

MODE = "NoEpisodicLife_NoClipReward_lr_1e-3_5e-3"
PATH = "WEIGHTS/" + game + "/" + MODE + "/"
print(PATH)

# Actor Network und Critic Network
Hier wird die Kernkomponente des Agenten definiert, also das künstliche neuronale Netz. Zu Beginn speichern wir uns Parameter ab, die bei dem Erstellen des Netzes wichtig sind. 

- Der Parameter "INPUT_SHAPE" speichert die Dimension der Eingabe, also dem von der Umgebung erhaltenen Bild. Die Vier am Ende steht dafür, dass wir vier Bilder aufeinandergestapelt eingeben. Die Dimension der Eingabe hängt von den definierten Parametern im FrameStack Wrapper und WarpFrame Wrapper ab, da diese die Bildeingabe umformen.

- Mit ACTOR_OUTPUT legen wir die Anzahl der Neuronen in der Ausgabeschicht des Actor-Netzwerkes fest. Das Actor-Netz braucht ein künstliches Neuron für jede gültige Aktion, die das Spiel zulässt. Mit env.action_space.n können wir die gültigen Aktionen der Umgebungsinstanz abfragen.

- CRITIC_OUTPUT legt die Anzahl der künstlichen Neuronen in der Ausgabeschicht des Critic-Netzwerkes fest. Das Critic-Netzwerk braucht nur ein Ausgabeneuron, da es mit diesem eine Abschätzung über die Güte des aktuellen Zustandes trifft.

- Zu jeweils beiden Netzwerken wird eine Lernrate definiert, die die Schrittgröße des Gradientenabstiegs während des Trainings der Netze bestimmt.


Im nächsten Schritt werden die neuronalen Netze bausteinartig zusammengesetzt. Mit net_input definieren wir die Eingabedimension, die die Eingabeschichten annehmen sollen. Conv2D sind Faltungsschichten, die sich besonders gut dafür eignen Bilder zu verarbeiten.
- Der Parameter filters gibt an, wie viele Filter in der Faltungsschicht zur Verarbeitung der Daten genutzt werden sollen.
- kernel_size bestimmt die Größe der eingesetzten Filter.
- strides beschreibt die Schrittweite, die ein Filter über die Eingabe verschoben wird.
- padding legt fest, ob die ursprüngleiche Dimensionalität der Eingaben beibehalten werden soll.

Nach jedem instanzierten Schichtaufruf wird die Eingabe der Schicht angegben. Beispielsweise werden unsere Schichten der Variable x zugewiesen und der nächsten Schicht übergeben. Da immer erst der linke Teil eines Gleicheheitszeichen aufgerufen wird, müssen wir nicht für jede unserer Schichten eine neue Variable definieren.

Nach jeder Schicht wird mit einer Aktivierungsfunktion die Aktivierung der Neuronen in einer Schicht berechnet. Die gängigste Aktivierungsfuntion ist dabei die ReLU-Aktivierungsfunktion.

Mit dem Aufruf von Flatten werden die berechneten Daten in ein Vektorformat gebracht, sodass sie von einer Dense-Schicht weiterverarbeitet werden können. Dense Schichten sind die "einfachsten" Schichten eines neuronalen Netzes und sind in der Literatur unter Feedfroward-Schicht zu finden. Der Parameter units gibt an, wie viele künstliche Neuronen sich in einer Dense-Schicht befinden sollen.

Der Aufbau der Netze wird an der Ausgabeschicht aufgeteilt. Beide bekommen für den Parameter "units" jeweils den vorher definierten Parameter ACTOR_OUTPUT oder CRITIC_OUTPUT übergeben.
Das Actor-Netzwerk nutzt die SoftMax-Verteilung als Aktivierungsfunktion an der Ausgabeschicht. Die SofMAx-Verteilung eignet sich zur Berechnung von Wahrscheinlichkeiten über eine gegebene Anzahl von Klassen. (Hier Aktionen)
Das Critic-Netzwerk bekommt als Aktivierungsfunktion "linear" übergeben. Das ist mit dem Vergeben keiner Aktivierungsfunktion gleichzusetzen, da so der "rohe" berecnete Wert des Netzes ausgegben wird. Die Ausgabe beschreibt die Bewertung des eingegebenen Zustand.

Zum Schluss instanziieren wir das neurnale Netz mit Model() und geben dabei die Eingabe- und die Ausgabeschicht an.

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Network Parameter
INPUT_SHAPE = (84, 84, 4) # (Höhe, Breite, Frames)
ACTOR_OUTPUT = env.action_space.n # Anzahl der möglichen Aktionen
CRITIC_OUTPUT = 1 # Bewertung der gewählten Aktion
ACTOR_LEARNING_RATE = 25e-6
CRITIC_LEARNING_RATE = 25e-6

# neuronales Netz
net_input = Input(shape=INPUT_SHAPE)
x = Conv2D(filters=32, kernel_size=(8, 8), strides=(4, 4), padding="same")(net_input)
x = Activation("relu")(x)
x = Conv2D(filters=64, kernel_size=(4, 4), strides=(2, 2), padding="same")(x)
x = Activation("relu")(x)
x = Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding="same")(x)
x = Activation("relu")(x)
x = Flatten()(x)
x = Dense(units=512)(x)
x = Activation("relu")(x)


'''Aufspalten des Netzes in Actor und Critic'''

""" Actor - wählt eine Aktion """ 
actor_x = Dense(units=ACTOR_OUTPUT, kernel_initializer='he_uniform')(x)
actor_output = Activation("softmax")(actor_x)

ACTOR = Model(inputs=net_input, outputs=actor_output)
ACTOR.compile(loss="categorical_crossentropy", optimizer=Adam(lr=ACTOR_LEARNING_RATE))

""" Critic - bewertet gewählte Aktion """
critic_x = Dense(units=CRITIC_OUTPUT)(x)
critic_output = Activation("linear")(critic_x)

CRITIC = Model(inputs=net_input, outputs=critic_output)
CRITIC.compile(loss="mse", optimizer=Adam(lr=CRITIC_LEARNING_RATE))

## Aktion wählen
Das Faltungsnetzwerk berechnet zu jedem Zustand eine Wahrscheinlichkeitsverteilung über die gültigen Aktionen. Die Wahl der auszuführenden Aktion wird von dieser Wahrscheinlichkeitsverteilung beeinflusst.

In [ ]:
def get_action(state):
    """ Berechnen einer Wahrscheinlichkeitsverteilung durch das Actor-Netzwerk"""
    policy = ACTOR.predict(state)[0]
    
    """ Wählen einer Aktion, die durch die berechnete Wahrscheinlichkeitsverteilung beeinflusst wird"""
    action = np.random.choice(env.action_space.n, p=policy)
    return action

Hier werden die gesammelten Erfahrungen einer Episode in Listen gespeichert.

In [ ]:
def remember (state, action, reward):
    STATES.append(state)
    """Erstellen eines One Hot Labels für die Aktion"""
    action_onehot = np.zeros([env.action_space.n])
    action_onehot[action] = 1
    ACTIONS.append(action_onehot)
    REWARDS.append(reward)

## Discount Rewards
Für jeden durchlaufenen Zustand, in einer Episode, wird der diskontierte Reward berechnet. Das diskontieren der Zustände beeinflusst den Agenten bei der Aktionswahl. Je höher der discount-Faktor GAMMA definiert wird, desto stärker bezieht der Agent in der Zukunft liegende Rewards in seiner Aktionswahl mit ein.

In [ ]:
def discount_rewards(rewards):
    gamma = 0.99
    running_add = 0 # Reward bis zu einem Zustand aus der Liste
    
    """
        anlegen einer leeren Liste, in der später die diskontierten 
        Rewards gespeichert werden
    """
    discounted_r = np.zeros_like(rewards)
    
    """ Berechnung des diskontierten Rewards für einen Zustand """
    for i in reversed(range(0, len(rewards))):
        running_add = running_add * gamma + rewards[i]
        discounted_r[i] = running_add

    discounted_r -= np.mean(discounted_r)
    discounted_r /= np.std(discounted_r)

    return discounted_r

# Replay Episodes
Zu Beginn werden die diskontierten Rewards einer Episode berechnet. Das Critic-Netzwerk berechnet eine Bewertung für die in der Episode durchlaufenden Zustände, die in der Vairable "values" abgespeichert werden.

Die Advantage-Werte ergeben sich dann aus der Differenz der diskontierten Rewards (zu erwartender Gesamtreward von einem Zustand aus) und der vom Critic-Netzwerk berechneten Bewertung.

Danach werden beide Netze trainiert. Das Actor-Netzwerk bekommt die gewählten Aktionen als Zielwerte übergeben. Die diskontierten Rewards werden dem Actor-Netzwerk zusätzlich als Wichtung für jedes Datenpaar mit übergeben, um die Aktionswahl
in Richtung des höchsten diskontierten Rewards zu drängen.
Das Crtic-Netzwerk bekommt als Zielwerte die diskontierten Rewards übergeben. Diese stellen den zu erwartenden Reward für einen Zustand dar und sind somit ein gutes Maß zur Zustandsbewertung.

In [ ]:
def replay_episode(STATES, ACTIONS, REWARDS):
    """ Daten in ein für das neuronale Netz kompatibles Format bringen"""
    states = np.vstack(STATES)
    actions = np.vstack(ACTIONS)

    """ Daten in ein für das neuronale Netz kompatibles Format bringen"""
    discounted_r = discount_rewards(REWARDS)

    """ Bewertung der Zustände durch das Critic-Netzwerk"""
    values = CRITIC.predict(states)[:, 0]

    """ berechnen der Advantage-Werte aus den diskontierten Rewards und den Zustandsbewertungen """
    advantages = discounted_r - values

    """ Trainieren der Netzwerke ACTOR und CRITIC"""
    ACTOR.fit(states, actions, sample_weight=advantages, epochs=1, verbose=0)
    CRITIC.fit(states, discounted_r, epochs=1, verbose=0)

    """ leeren des Episoden Buffers"""
    STATES, ACTIONS, REWARDS = [], [], []

# Training /Spielen

In [ ]:
""" 
Speichern eines initialen Mean Rewards. Während des Trainings wird der Mean Reward aus den 
letzten zehn gespielten Episoden berechnet. Ist der berechnete Mean Reward besser, so werden die
Netzparameter gespeichert und der Mean Reward mit dem aktuellen überschrieben.

Der initiale Mean Reward kann nicht auf 0 gesetzt werden, da der Score des Spiels Pong bei -21 startet.
Durch Spielen einer zufälligenen Episode samplen wir einen Beispielhaften aber schlechten Reward zum Beginnen.
"""
INITIAL_MEAN_REWARD = 0.0
env.reset()
while True:
    _, reward, done, _ = env.step(env.action_space.sample())
    INITIAL_MEAN_REWARD += reward
    if done:
        break
INITIAL_MEAN_REWARD

Zu Beginn legen wir die Episodenanzahl fest, die der Agent zum Training durchlaufen soll.
Die verschiedenen Listen werden angelegt, um Parameter abzuspeichern, an denen das Training abschließend bewertet werden kann.

Wir starten den Trainingsprozess mit einer for-Schleife für EPISODES viele Iterationen. Zu Beginn jeder Iteration wird das Spiel neu gestartet, da eine Episode pro Iteration gespielt wird. Die "done"-Flag wird auf False gesetzt, da wir uns nicht länger in einem Endzustand befinden.

Die while-Schleife leitet dann eine Episode ein und wird so lange durchlaufen, bis die "done"-Flag das Ende einer Episode signalisiert.

Zu Beginn jeder Episode wählen wir eine Aktion auf Grundlage des Initialzustandes (state = env.reset()). Während der Episode werden die Aktionen anschließend anhand des aktuellen Episodenzustandes gewählt (next_state, ... = env.step(action)).
Durch das Interagieren mit der Umgebung erhält der Agent eine neue Beobachtung (next_state), eine Belohnung (reward), eine Flag (done) und ein info-Dictionary (_) von der Umgebung. Das info-Dictionary enhält weitere Informationen, wie beispielsweise die Leben des Agenten, falls vorhanden. Das Dictionary wird nicht für das Training des Agenten benötigt.

Die erhaltenen Parameter werden mit remember() in Listen abgespeichert und werden nach jeder Episode für das Training verwendet.

Das Training des neuronalen Netzes wird durch die Methode "replay()" eingeleitet.

In [ ]:
EPISODES = 15_000
REWARD_LIST = []
MEAN_LIST = []
BEST_MEAN_REWARD = INITIAL_MEAN_REWARD

# for-Schleife des Trainingsprozesses
for episode in range(EPISODES):
    EPISODE_REWARD = 0.0
    state = env.reset()
    done = False

    # Episoden Buffer
    STATES, ACTIONS, REWARDS = [], [], []
    
    # while-Schleife einer Episode
    while not done:
        action = get_action(state)
        next_state, reward, done, _ = env.step(action)
        
        """ speichern einer Transition im Episodenspeicher """
        remember(state, action, reward)

        """ aktualisieren des state Variable auf den neusten Umgebungszustand """
        state = next_state
        
        """ Reward einer Aktion zum gesamten Reward der Episode addieren """
        EPISODE_REWARD += reward


        if done:
            REWARD_LIST.append(EPISODE_REWARD)
            current_mean_reward = np.mean(REWARD_LIST[-min(len(REWARD_LIST), 10):])
            MEAN_LIST.append(np.mean(REWARD_LIST))
            
            """ Ausgabe des aktuellen Trainingsfortschrittes """
            print("Episode:", episode+1, "\tReward:", EPISODE_REWARD, "\tMean:", round(current_mean_reward, 2),"\tBestMean:", BEST_MEAN_REWARD)

            """ Übernahme des höchsten Mean Rewards """
            if current_mean_reward > BEST_MEAN_REWARD:
                BEST_MEAN_REWARD = current_mean_reward
        
                """ Trainierte Gewichte speichern """
                import os
                try:
                    os.makedirs(PATH)
                except FileExistsError:
                    # Pfad existiert bereits
                    pass
                ACTOR.save_weights(PATH + "Best_ACTOR.h5")
                CRITIC.save_weights(PATH + "Best_CRITIC.h5")

            replay_episode(STATES, ACTIONS, REWARDS)

In [ ]:
import pandas as pd
from datetime import datetime
date = datetime.now().date()

df = pd.DataFrame(list(zip(REWARD_LIST, MEAN_LIST)), 
               columns =['Rewards', 'Mean Reward']) 
df.to_csv(PATH + game + "_" + str(date) + "_"+ MODE + ".csv", mode="w", index=False)

In [ ]:
ACTOR.save_weights(PATH + "End.h5")
CRITIC.save_weights(PATH + "End.h5")

# Auswertung

In [ ]:
""" Erstellen einer Grafik über den Trainingsverlauf """
plt.figure(figsize=(25, 12))
plt.plot(REWARD_LIST, label="erhaltene Rewards")
plt.plot(MEAN_LIST, label="durchschnittler Reward")
plt.title("Rewards während des Trainings", fontsize=25)
plt.xlabel("Episoden", fontsize=20)
plt.ylabel("Rewards", fontsize=20)
plt.legend(fontsize=15)
plt.show()

# Spielen

In [ ]:
# Gewichte laden
#WEIGHTS_PATH = "WEIGHTS/...h5"
#ACTOR.load_weights(filepath=WEIGHTS_PATH)

In [ ]:
import gym
from IPython import display
import matplotlib

for i in range(1):
    state = env.reset()
    done = False
    while not done:
        img = plt.imshow(env.render(mode='rgb_array'))
        img.set_data(env.render(mode='rgb_array'))
        display.display(plt.gcf())
        display.clear_output(wait=True)
        action = np.argmax(ACTOR.predict(state))
        state, reward, done, info = env.step(action)